In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)

import geopandas as gpd
from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
log = logging.getLogger(__name__)

In [2]:
# change to your custom locations
data_dir = "/data/S1"
ids = ['S1A_IW_SLC__1SDV_20181228T050448_20181228T050515_025221_02C9BE_1E22']
primary_path = f"{data_dir}/{ids[0]}.zip"
output_dir_flattened="/data/res/etna-flattened"
output_dir_beta="/data/res/etna-beta"

## Download S-1 products

In [ ]:
# load a geometry
# aoi_file = "../data/Etna.geojson"
aoi_file = "https://raw.githubusercontent.com/odhondt/eo_tools/refs/heads/main/data/Etna.geojson"
shp = gpd.read_file(aoi_file).geometry[0]

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2018-12-22",
    "end": "2018-12-29",
    "geom": shp
}

results = dag.search(**search_criteria)
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, output_dir="/data/S1/", extract=False)

## Geocoding with terrain flattening

In [ ]:
from eo_tools.S1.process import process_slc

out_dir_flattened = process_slc(
    slc_path=primary_path,
    output_dir=output_dir_flattened,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    multilook=[1, 4],
    warp_kernel="bicubic",
    cal_type="terrain",
    clip_to_shape=True,
)

## Geocoding with beta nought for comparison

In [ ]:
from eo_tools.S1.process import process_slc

out_dir_beta = process_slc(
    slc_path=primary_path,
    output_dir=output_dir_beta,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    multilook=[1, 4],
    warp_kernel="bicubic",
    cal_type="beta",
    clip_to_shape=True,
)

# Display results

In [ ]:
from folium import Map, LayerControl
from eo_tools_dev.util import show_cog

m = Map()
_ = show_cog(f"{out_dir_beta}/amp_vv.tif", m, rescale=f"0, 1")
_ = show_cog(f"{out_dir_flattened}/amp_vv.tif", m, rescale=f"0, 1")
LayerControl().add_to(m)
m